
# Learning Series #3 — Survival & Prognostic Biomarker Tools (TCGA)

This notebook evaluates survival differences based on gene expression (e.g., **CD274** / PD-L1) using Kaplan–Meier curves and Cox models.

> **Tip:** Keep your variable names clean and consistent (e.g., `OS_time`, `OS_event`).



## 0) Environment
Install required libraries if needed.


In [ ]:

# If needed, uncomment:
# !pip install pandas numpy lifelines matplotlib scipy statsmodels



## 1) Imports & Settings


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter

# Plot settings
plt.rcParams["figure.dpi"] = 140
plt.rcParams["savefig.bbox"] = "tight"

print("Versions:")
import sys; print("Python:", sys.version.split()[0])
import lifelines, matplotlib
print("lifelines:", lifelines.__version__)
print("matplotlib:", matplotlib.__version__)



## 2) Load Clinical & Expression Data

- `clinical_path` must include at least: `sample_id`, `OS_time`, `OS_event`.
- `expr_path` must be a matrix: row = samples, columns = genes; first column `sample_id`.


In [ ]:

clinical_path = "example_clinical.csv"   # change to your file
expr_path = "example_expression.csv"     # change to your file

clinical = pd.read_csv(clinical_path)
expr = pd.read_csv(expr_path)

# Basic checks
assert "sample_id" in clinical.columns, "clinical must have 'sample_id'"
assert "OS_time" in clinical.columns and "OS_event" in clinical.columns, "clinical needs OS_time & OS_event"
assert "sample_id" in expr.columns, "expression must have 'sample_id' column"

# Merge on sample_id (inner join to keep matched samples)
df = clinical.merge(expr, on="sample_id", how="inner")
print("Merged shape:", df.shape)
df.head()



## 3) Choose Biomarker & Define Groups
Pick a gene symbol present in your expression matrix. We'll create high/low groups based on the median (or a custom cutoff).


In [ ]:

biomarker = "CD274"  # change to your gene of interest (e.g., "NKX2-1" for TTF1, "GATA3", "MKI67")
assert biomarker in df.columns, f"{biomarker} not found in expression matrix"

# Cut method: 'median' (default) or numeric value (e.g., 5.0 TPM/FPKM/log2 CPM unit)
cut_method = "median"

if cut_method == "median":
    cutoff = df[biomarker].median()
else:
    cutoff = float(cut_method)

df["group"] = np.where(df[biomarker] >= cutoff, "High", "Low")
df["group"].value_counts()



## 4) Kaplan–Meier Survival Curves + Log-rank Test


In [ ]:

time_col = "OS_time"     # ensure units are consistent (days vs months)
event_col = "OS_event"

kmf = KaplanMeierFitter()

fig = plt.figure(figsize=(5,4))
for label in ["High", "Low"]:
    mask = df["group"] == label
    kmf.fit(df.loc[mask, time_col], df.loc[mask, event_col], label=label)
    kmf.plot_survival_function()  # do not set specific colors per instructions

plt.title(f"KM: {biomarker} High vs Low")
plt.xlabel("Time")
plt.ylabel("Survival probability")
plt.grid(True, alpha=0.3)
plt.show()

# Log-rank test
high = df[df["group"] == "High"]
low  = df[df["group"] == "Low"]
res = logrank_test(high[time_col], low[time_col], event_observed_A=high[event_col], event_observed_B=low[event_col])
print("Log-rank p-value:", res.p_value)



## 5) Univariable & Multivariable Cox PH
We model hazard as a function of the biomarker (continuous) and optionally covariates (e.g., age, stage).


In [ ]:

cox = CoxPHFitter()

# Example covariates (add if present in your clinical data)
covariates = ["age", "grade", "stage"]  # edit to match your columns
existing_covs = [c for c in covariates if c in df.columns]

cox_df = df[[time_col, event_col, biomarker] + existing_covs].dropna()
cox_df = cox_df.rename(columns={time_col: "T", event_col: "E"})

cox.fit(cox_df, duration_col="T", event_col="E")
cox.print_summary()  # HR, CI, p-values

# Optional: simple forest-style visualization via summary table
summary = cox.summary[["coef", "exp(coef)", "p", "exp(coef) lower 95%", "exp(coef) upper 95%"]]
summary



## 6) Batch Analysis (Multiple Markers)
Loop over a gene list to compute univariable HRs and make a compact forest plot.


In [ ]:

gene_list = ["CD274", "GATA3", "NKX2-1"]  # change as needed
rows = []

for g in gene_list:
    if g not in df.columns: 
        continue
    tmp = df[[time_col, event_col, g]].dropna().rename(columns={time_col: "T", event_col: "E"})
    if tmp[g].nunique() < 3:
        continue
    c = CoxPHFitter()
    try:
        c.fit(tmp, duration_col="T", event_col="E")
        s = c.summary.loc[g, :]
        rows.append({
            "gene": g,
            "HR": s["exp(coef)"],
            "HR_lower": s["exp(coef) lower 95%"],
            "HR_upper": s["exp(coef) upper 95%"],
            "p": s["p"]
        })
    except Exception as e:
        print(f"Skipping {g}: {e}")

hr_df = pd.DataFrame(rows).sort_values("HR")
hr_df


In [ ]:

# Simple forest plot
if not hr_df.empty:
    fig = plt.figure(figsize=(6, 0.5*len(hr_df) + 1))
    y = np.arange(len(hr_df))
    plt.errorbar(hr_df["HR"], y, xerr=[hr_df["HR"]-hr_df["HR_lower"], hr_df["HR_upper"]-hr_df["HR"]], fmt='o')
    plt.axvline(1.0, linestyle="--")
    plt.yticks(y, hr_df["gene"])
    plt.xlabel("Hazard Ratio (HR)")
    plt.title("Univariable Cox HRs")
    plt.grid(True, axis="x", alpha=0.3)
    plt.tight_layout()
    plt.show()



## 7) Save Outputs


In [ ]:

# Tables
out_table = "cox_summary_table.csv"
summary.to_csv(out_table, index=True)
print(f"Saved: {out_table}")

# Figures: re-run the plots above and call plt.savefig('filename.png') where needed.
# Example:
# plt.savefig("km_cd274_high_low.png", dpi=300)



## 8) Notes & Tips
- For TCGA, ensure consistent units for survival time (days/months) and that censoring is correct.
- Consider adding stage/grade as covariates for multivariable models.
- Try alternate cutoffs (tertiles, quartiles). For immunomarkers (e.g., PD-L1), consider clinically meaningful thresholds if known.
- Validate proportional hazards assumptions (e.g., with `lifelines` `check_assumptions`).

*Happy analyzing!* 👩‍⚕️🧬
